In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


# URL to the raw CSV file
url = 'https://raw.githubusercontent.com/Gifty0/report/main/food_data.csv'

# Read the CSV file directly from the URL with a specific encoding
df = pd.read_csv(url, encoding='latin1')

# Display the first few rows of the dataframe
print(df.head())


   NDB_No                 Shrt_Desc  Water_(g)  Energ_Kcal  Protein_(g)  \
0    1001              Energy Drink   5.037006  352.358157    15.041196   
1    1002               Cauliflower  69.909807  251.382142     3.948495   
2    1003                       Pie  99.239640  363.234726    36.343898   
3    1004              Clam Chowder  26.726254  220.246765    48.771207   
4    1005  New England Clam Chowder  67.909062   95.014422    39.104469   

   Lipid_Tot_(g)   Ash_(g)  Carbohydrt_(g)  Fiber_TD_(g)  Sugar_Tot_(g)  ...  \
0      24.919405  4.301456       12.963159      1.383696      16.082692  ...   
1      12.181176  4.312533        4.130635     17.334060       9.435268  ...   
2      49.895876  5.311951        0.474750      8.367269      43.684038  ...   
3      33.555779  4.706474       35.680508     18.496497      36.144461  ...   
4      44.188972  6.979938       24.253202     12.845587      48.903946  ...   

      Vit_D_IU  Vit_K_(ug)  FA_Sat_(g)  FA_Mono_(g)  FA_Poly_(g)  \


In [4]:
# Select relevant features for the model
features = [
    'Energ_Kcal', 'Protein_(g)', 'Lipid_Tot_(g)', 'Carbohydrt_(g)',
    'Fiber_TD_(g)', 'Sugar_Tot_(g)', 'Calcium_(mg)', 'Iron_(mg)',
    'Magnesium_(mg)', 'Phosphorus_(mg)', 'Potassium_(mg)', 'Sodium_(mg)',
    'Zinc_(mg)', 'Copper_(mg)', 'Manganese_(mg)', 'Selenium_(ug)',
    'Vit_C_(mg)', 'Thiamin_(mg)', 'Riboflavin_(mg)', 'Niacin_(mg)',
    'Panto_Acid_(mg)', 'Vit_B6_(mg)', 'Folate_Tot_(ug)', 'Choline_Tot_(mg)',
    'Vit_B12_(ug)', 'Vit_A_IU', 'Vit_E_(mg)', 'Vit_D_(ug)', 'Vit_K_(ug)',
    'FA_Sat_(g)', 'FA_Mono_(g)', 'FA_Poly_(g)', 'Cholestrl_(mg)'
]

# Extract features and target variable
X = df[features]
y = df['Energ_Kcal']  # Using calories as the target variable for this example

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


Model and training

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)

# Train a RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_train, y_train)

# Make predictions on the test
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 3.285624378020912


In [6]:
from ipywidgets import FloatText, Dropdown, IntText, Button, HBox, VBox, Output
from IPython.display import display, HTML

# Define the widgets
weight = FloatText(value=70.0, description='Weight (kg):')
height = FloatText(value=170.0, description='Height (cm):')
age = IntText(value=25, description='Age:')
gender = Dropdown(options=['Male', 'Female'], value='Male', description='Gender:')
activity_level = Dropdown(options=['Sedentary', 'Lightly active', 'Moderately active', 'Very active', 'Super active'],
                          value='Moderately active', description='Activity Level:')
goal = Dropdown(options=['lose', 'maintain', 'gain'], value='maintain', description='Goal:')
submit_button = Button(description='Submit')

# Display the widgets
display(VBox([weight, height, age, gender, activity_level, goal, submit_button]))

def calculate_caloric_needs(weight, height, age, gender, activity_level, goal):
    if gender == 'Male':
        bmr = 88.362 + (13.397 * weight) + (4.799 * height) - (5.677 * age)
    else:
        bmr = 447.593 + (9.247 * weight) + (3.098 * height) - (4.330 * age)

    activity_multiplier = {
        'Sedentary': 1.2,
        'Lightly active': 1.375,
        'Moderately active': 1.55,
        'Very active': 1.725,
        'Super active': 1.9
    }

    tdee = bmr * activity_multiplier[activity_level]

    if goal == 'lose':
        tdee -= 500  # Deduct 500 calories for weight loss
    elif goal == 'gain':
        tdee += 500  # Add 500 calories for weight gain

    return tdee

def on_submit_button_clicked(b):
    w = weight.value
    h = height.value
    a = age.value
    g = gender.value
    al = activity_level.value
    goal_value = goal.value

    tdee = calculate_caloric_needs(w, h, a, g, al, goal_value)

    # Display the calculated TDEE
    output.clear_output()
    with output:
        display(HTML(f'<h3>Daily Caloric Needs: {tdee:.2f} kcal</h3>'))



#simplify filtering
def on_submit_button_clicked(b):
    w = weight.value
    h = height.value
    a = age.value
    g = gender.value
    al = activity_level.value
    goal_value = goal.value

    tdee = calculate_caloric_needs(w, h, a, g, al, goal_value)
    protein_needs, carb_needs, fat_needs = calculate_macronutrient_needs(tdee, goal_value)

    # Display calculated needs for debugging
    output.clear_output()
    with output:
        display(HTML(f'<h3>Daily Caloric Needs: {tdee:.2f} kcal</h3>'))
        display(HTML(f'<h4>Protein Needs: {protein_needs:.2f} g, Carbs Needs: {carb_needs:.2f} g, Fat Needs: {fat_needs:.2f} g</h4>'))



output = Output()
submit_button.on_click(on_submit_button_clicked)

display(output)

       # Define a function to calculate daily macronutrient needs
def calculate_macronutrient_needs(tdee, goal):
    if goal == 'lose':
        protein = 0.4 * tdee / 4
        carbs = 0.4 * tdee / 4
        fats = 0.2 * tdee / 9
    elif goal == 'gain':
        protein = 0.3 * tdee / 4
        carbs = 0.5 * tdee / 4
        fats = 0.2 * tdee / 9
    else:  # maintain
        protein = 0.3 * tdee / 4
        carbs = 0.4 * tdee / 4
        fats = 0.3 * tdee / 9
    return protein, carbs, fats

def on_submit_button_clicked(b):
    w = weight.value
    h = height.value
    a = age.value
    g = gender.value
    al = activity_level.value
    goal_value = goal.value

    tdee = calculate_caloric_needs(w, h, a, g, al, goal_value)
    protein_needs, carb_needs, fat_needs = calculate_macronutrient_needs(tdee, goal_value)

    # Define ranges for filtering (±30% for more flexibility)
    cal_range = tdee * 0.3
    protein_range = protein_needs * 0.3
    carb_range = carb_needs * 0.3
    fat_range = fat_needs * 0.3

    # Display calculated needs and ranges for debugging
    output.clear_output()
    with output:
        display(HTML(f'<h3>Daily Caloric Needs: {tdee:.2f} kcal</h3>'))
        display(HTML(f'<h4>Protein Needs: {protein_needs:.2f} g, Carbs Needs: {carb_needs:.2f} g, Fat Needs: {fat_needs:.2f} g</h4>'))
        display(HTML(f'<h4>Caloric Range: {tdee - cal_range:.2f} - {tdee + cal_range:.2f} kcal</h4>'))
        display(HTML(f'<h4>Protein Range: {protein_needs - protein_range:.2f} - {protein_needs + protein_range:.2f} g</h4>'))
        display(HTML(f'<h4>Carbs Range: {carb_needs - carb_range:.2f} - {carb_needs + carb_range:.2f} g</h4>'))
        display(HTML(f'<h4>Fat Range: {fat_needs - fat_range:.2f} - {fat_needs + fat_range:.2f} g</h4>'))





Output()

In [7]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.display import display, HTML
import time
import random
from IPython.display import display, HTML, clear_output, Javascript


# Initialize scaler and model
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[features])
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X_scaled, df['Energ_Kcal'])

def recommend_foods(user_weight, goal):
    # Filter foods that meet caloric and protein requirements based on goal
    if goal == 'lose':
        calories_needed = user_weight * 20
    elif goal == 'maintain':
        calories_needed = user_weight * 25
    elif goal == 'gain':
        calories_needed = user_weight * 30
    else:
        raise ValueError("Goal should be 'lose', 'maintain', or 'gain'")

    # Predict calories and protein for each food item
    df['Predicted_Calories'] = model.predict(X_scaled)
    df['Predicted_Protein'] = df['Protein_(g)']

    # Calculate portion size in grams based on calories per 100g
    df['Portion_Size_g'] = (100 * calories_needed) / df['Energ_Kcal']

    # Filter foods that meet caloric and protein requirements
    filtered_foods = df[(df['Predicted_Calories'] <= calories_needed) & (df['Predicted_Protein'] >= 10)]

    # Select a diverse set of food types based on Shrt_Desc
    sampled_foods = filtered_foods.groupby('Shrt_Desc').apply(lambda x: x.sample(1)).reset_index(drop=True)
    sampled_foods = sampled_foods[['Shrt_Desc', 'Predicted_Calories', 'Predicted_Protein', 'Portion_Size_g']]

    # Shuffle the sampled foods
    sampled_foods = sampled_foods.sample(frac=1).reset_index(drop=True)

    return sampled_foods

def animate_loading():
    # Define loading animation elements
    animations = [
        "<h3>Loading...</h3>",
        "<h3>Processing your request...</h3>",
        "<h3>Hang tight, we're working on it...</h3>"
    ]
    # Randomly select and display an animation
    animation = np.random.choice(animations)
    display(widgets.HTML(animation))

def show_popup(message):
    display(Javascript(f'alert("{message}");'))

def on_submit_button_clicked(b):
    user_weight = weight_input.value
    goal = goal_input.value.lower()

    # Validate user input
    if user_weight <= 0:
        show_popup("Please enter a valid weight (greater than zero).")
        return

    # Clear previous output and display loading animation
    clear_output()
    display(weight_input, goal_input, submit_button)
    animate_loading()

    # Simulate delay for demonstration (replace with actual calculations)
    time.sleep(2)

    # Get recommendations
    recommendations = recommend_foods(user_weight, goal)

    # Generate HTML for recommendations
    if not recommendations.empty:
        html_output = "<h3>Recommended Foods:</h3>"
        html_output += recommendations.head(10).to_html(index=False)
    else:
        html_output = "<p>No foods found matching your criteria.</p>"

    # Clear loading animation and display recommendations
    clear_output()
    display(weight_input, goal_input, submit_button)
    display(widgets.HTML(html_output))

# Create input widgets
weight_input = widgets.FloatText(description='Weight (kg):')
goal_input = widgets.Dropdown(options=['lose', 'maintain', 'gain'], description='Goal:')
submit_button = widgets.Button(description='Submit')

# Display widgets
display(weight_input, goal_input, submit_button)

# Link button click event to function
submit_button.on_click(on_submit_button_clicked)

FloatText(value=65.0, description='Weight (kg):')

Dropdown(description='Goal:', index=1, options=('lose', 'maintain', 'gain'), value='maintain')

Button(description='Submit', style=ButtonStyle())

HTML(value='<h3>Recommended Foods:</h3><table border="1" class="dataframe">\n  <thead>\n    <tr style="text-al…